In [2]:
import pandas as pd
import kagglehub
import shutil
import os
import ast

/Users/vikram/miniconda3/envs/ml-0451/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Download the dataset
path = kagglehub.dataset_download("rounakbanik/the-movies-dataset")

# Define current directory
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, "data")

# Copy all files from the kagglehub cache folder to current dir
for filename in os.listdir(path):
    src = os.path.join(path, filename)
    dst = os.path.join(data_dir, filename)
    if os.path.isfile(src):
        shutil.copy(src, dst)

print(f"All files copied to: {data_dir}")
os.rename("../data/keywords.csv", "../data/movies_keywords.csv")


All files copied to: /Users/vikram/Desktop/Year 4/Semester 2/Machine Learning/Project/cs451-project/data


In [ ]:
# Function to extract genre names
def extract_comma_separated(df, column_name, key='name'):
    """
    Converts stringified lists of dicts in a column into comma-separated values of a given key.
    
    Args:
        df (pd.DataFrame): Your DataFrame
        column_name (str): The name of the column to process
        key (str): The key to extract from each dictionary in the list
    
    Returns:
        pd.Series: A cleaned column with comma-separated strings
    """
    def extract_values(obj_str):
        try:
            items = ast.literal_eval(obj_str)
            return ', '.join([str(item[key]) for item in items if key in item])
        except (ValueError, SyntaxError, TypeError):
            return None

    return df[column_name].apply(extract_values)

def extract_single_dict_value(obj_str, key='name'):
    """
    Extracts the value of a given key from a stringified dictionary.

    Args:
        obj_str (str): The stringified dictionary
        key (str): The key to extract

    Returns:
        str or None: The value corresponding to the key, or None if invalid
    """
    try:
        obj = ast.literal_eval(obj_str)
        return str(obj.get(key))
    except (ValueError, SyntaxError, AttributeError):
        return None
    
# Load your dataset
df = pd.read_csv("../data/movies_metadata.csv", low_memory=False)
metadata = df.copy()
print(f"Inital Count: {metadata.shape[0]}")
metadata = metadata[metadata['status'] == 'Released']
print(f"Released Count: {metadata.shape[0]}")
metadata = metadata[metadata['vote_count'] >= 100]
print(f"Sufficient Votes Count: {metadata.shape[0]}")

# Apply the function to the 'genres' column
metadata['collection_name'] = df['belongs_to_collection'].apply(lambda x: extract_single_dict_value(x, 'name'))
metadata['collection_id'] = df['belongs_to_collection'].apply(lambda x: extract_single_dict_value(x, 'id'))
metadata['genres'] = extract_comma_separated(df, 'genres', key='name')
metadata['genre_ids'] = extract_comma_separated(df, 'genres', key='id')
metadata['production_companies'] = extract_comma_separated(df, 'production_companies', key='name')
metadata['production_companies_ids'] = extract_comma_separated(df, 'production_companies', key='id')
metadata['production_countries'] = extract_comma_separated(df, 'production_countries', key='iso_3166_1')
metadata['spoken_languages'] = extract_comma_separated(df, 'spoken_languages', key='name')
metadata.drop(columns=['belongs_to_collection', 'homepage', 'imdb_id', 'original_title', 'overview', 'poster_path', 'status', 'tagline', 'video'], inplace=True)

# Preview result
display(metadata.head())

Inital Count: 45466
Released Count: 45014
Sufficient Votes Count: 6052


,adult,budget,genres,id,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count,collection_name,collection_id,genre_ids,production_companies_ids
0,False,30000000,"Animation, Comedy, Family",862,en,21.946943,Pixar Animation Studios,US,1995-10-30,373554033.0,81.0,English,Toy Story,7.7,5415.0,Toy Story Collection,10194,"16, 35, 10751",3
1,False,65000000,"Adventure, Fantasy, Family",8844,en,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",US,1995-12-15,262797249.0,104.0,"English, Français",Jumanji,6.9,2413.0,None,None,"12, 14, 10751","559, 2550, 10201"
4,False,0,Comedy,11862,en,8.387519,"Sandollar Productions, Touchstone Pictures",US,1995-02-10,76578911.0,106.0,English,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,96871,35,"5842, 9195"
5,False,60000000,"Action, Crime, Drama, Thriller",949,en,17.924927,"Regency Enterprises, Forward Pass, Warner Bros.",US,1995-12-15,187436818.0,170.0,"English, Español",Heat,7.7,1886.0,None,None,"28, 80, 18, 53","508, 675, 6194"
6,False,58000000,"Comedy, Romance",11860,en,6.677277,"Paramount Pictures, Scott Rudin Productions, M...","DE, US",1995-12-15,0.0,127.0,"Français, English",Sabrina,6.2,141.0,None,None,"35, 10749","4, 258, 932, 5842, 14941, 55873, 58079"


In [ ]:
# Load the DataFrame
df = pd.read_csv("../data/credits.csv")

# Define a function to simplify and convert types
def simplify_people(credits_str):
    try:
        return ast.literal_eval(credits_str)
    except (ValueError, SyntaxError):
        return []

# Apply to both 'cast' and 'crew'
credits = pd.DataFrame()
credits['id'] = df['id'].astype(str)
credits['cast'] = df['cast'].apply(simplify_people)
credits['crew'] = df['crew'].apply(simplify_people)

# Extract all of the cast members
credits['all_cast'] = credits['cast'].apply(
    lambda x: [{'name': member['name'], 'gender': str(member['gender']), 'id': str(member['id'])} for member in x]
)

# Updated function to extract director info
def extract_director_info(crew_list):
    for person in crew_list:
        if person.get('job') == 'Director':
            return {
                'name': person.get('name'),
                'gender': str(person.get('gender')),
                'id': str(person.get('id'))
            }
    return {'name': None, 'gender': None, 'id': None}

# Apply the function
credits['director'] = credits['crew'].apply(extract_director_info)
credits.drop(columns=['cast', 'crew'], inplace=True)

credits.head()


,id,all_cast,director
0,862,"[{'name': 'Tom Hanks', 'gender': '2', 'id': '3...","{'name': 'John Lasseter', 'gender': '2', 'id':..."
1,8844,"[{'name': 'Robin Williams', 'gender': '2', 'id...","{'name': 'Joe Johnston', 'gender': '2', 'id': ..."
2,15602,"[{'name': 'Walter Matthau', 'gender': '2', 'id...","{'name': 'Howard Deutch', 'gender': '2', 'id':..."
3,31357,"[{'name': 'Whitney Houston', 'gender': '1', 'i...","{'name': 'Forest Whitaker', 'gender': '2', 'id..."
4,11862,"[{'name': 'Steve Martin', 'gender': '2', 'id':...","{'name': 'Charles Shyer', 'gender': '2', 'id':..."


In [ ]:
# Load the DataFrame
df = pd.read_csv("../data/movies_keywords.csv")

# Define a function to simplify and convert types
def simplify_keywords(keyword_str):
    try:
        return ast.literal_eval(keyword_str)
    except (ValueError, SyntaxError):
        return []

# Apply to keywords
keywords = pd.DataFrame()
keywords['id'] = df['id'].astype(str)
keywords['keywords'] = df['keywords'].apply(simplify_keywords)

# extract all of the keywords
keywords['all_keywords'] = keywords['keywords'].apply(
    lambda x: [{'name': keyword['name'], 'id': str(keyword['id'])} for keyword in x]
)

# Drop the original 'cast', 'crew', and 'director_info' columns
keywords = keywords.drop(columns=['keywords'])

keywords.head()


,id,all_keywords
0,862,"[{'name': 'jealousy', 'id': '931'}, {'name': '..."
1,8844,"[{'name': 'board game', 'id': '10090'}, {'name..."
2,15602,"[{'name': 'fishing', 'id': '1495'}, {'name': '..."
3,31357,"[{'name': 'based on novel', 'id': '818'}, {'na..."
4,11862,"[{'name': 'baby', 'id': '1009'}, {'name': 'mid..."


In [ ]:
# Merge the two DataFrames on 'id'
merged_df = pd.merge(metadata, credits, on='id', how='left')
merged_df = pd.merge(merged_df, keywords, on='id', how='left')

display(merged_df.head())

# Save the merged DataFrame to a CSV file
merged_df.to_csv("../clean_data/movies.csv", index=False)

,adult,budget,genres,id,original_language,popularity,production_companies,production_countries,release_date,revenue,...,title,vote_average,vote_count,collection_name,collection_id,genre_ids,production_companies_ids,all_cast,director,all_keywords
0,False,30000000,"Animation, Comedy, Family",862,en,21.946943,Pixar Animation Studios,US,1995-10-30,373554033.0,...,Toy Story,7.7,5415.0,Toy Story Collection,10194,"16, 35, 10751",3,"[{'name': 'Tom Hanks', 'gender': '2', 'id': '3...","{'name': 'John Lasseter', 'gender': '2', 'id':...","[{'name': 'jealousy', 'id': '931'}, {'name': '..."
1,False,65000000,"Adventure, Fantasy, Family",8844,en,17.015539,"TriStar Pictures, Teitler Film, Interscope Com...",US,1995-12-15,262797249.0,...,Jumanji,6.9,2413.0,None,None,"12, 14, 10751","559, 2550, 10201","[{'name': 'Robin Williams', 'gender': '2', 'id...","{'name': 'Joe Johnston', 'gender': '2', 'id': ...","[{'name': 'board game', 'id': '10090'}, {'name..."
2,False,0,Comedy,11862,en,8.387519,"Sandollar Productions, Touchstone Pictures",US,1995-02-10,76578911.0,...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,96871,35,"5842, 9195","[{'name': 'Steve Martin', 'gender': '2', 'id':...","{'name': 'Charles Shyer', 'gender': '2', 'id':...","[{'name': 'baby', 'id': '1009'}, {'name': 'mid..."
3,False,60000000,"Action, Crime, Drama, Thriller",949,en,17.924927,"Regency Enterprises, Forward Pass, Warner Bros.",US,1995-12-15,187436818.0,...,Heat,7.7,1886.0,None,None,"28, 80, 18, 53","508, 675, 6194","[{'name': 'Al Pacino', 'gender': '2', 'id': '1...","{'name': 'Michael Mann', 'gender': '2', 'id': ...","[{'name': 'robbery', 'id': '642'}, {'name': 'd..."
4,False,58000000,"Comedy, Romance",11860,en,6.677277,"Paramount Pictures, Scott Rudin Productions, M...","DE, US",1995-12-15,0.0,...,Sabrina,6.2,141.0,None,None,"35, 10749","4, 258, 932, 5842, 14941, 55873, 58079","[{'name': 'Harrison Ford', 'gender': '2', 'id'...","{'name': 'Sydney Pollack', 'gender': '2', 'id'...","[{'name': 'paris', 'id': '90'}, {'name': 'brot..."


In [11]:
# Load your credits CSV
df = pd.read_csv("data/credits.csv")

# Define a function to extract name, id, and gender from cast JSON
def extract_cast_info(cast_json_str):
    try:
        cast_list = ast.literal_eval(cast_json_str)
        return [{'name': member['name'], 'id': str(member['id']), 'gender': str(member['gender'])} for member in cast_list]
    except:
        return []
    
# Define a function to extract director info from crew JSON
def extract_director_info(crew_json_str):
    try:
        crew_list = ast.literal_eval(crew_json_str)
        return [{'name': member['name'], 'id': str(member['id']), 'gender': str(member['gender'])}
                for member in crew_list if member.get('job') == 'Director']
    except:
        return []

# Apply the function to the 'cast' column
df['simplified_cast'] = df['cast'].apply(extract_cast_info)
# Apply the function to the 'crew' column
df['simplified_directors'] = df['crew'].apply(extract_director_info)

# explode to have one row per cast member (with name, id, gender)
exploded_actors_df = df.explode('simplified_cast')
exploded_actors_df = pd.json_normalize(exploded_actors_df['simplified_cast'])

# explode to have one row per director (with name, id, gender)
exploded_directors_df = df.explode('simplified_directors')
exploded_directors_df = pd.json_normalize(exploded_directors_df['simplified_directors'])

# Show result
display(exploded_actors_df.head())
display(exploded_directors_df.head())

# Save the exploded DataFrame to a CSV file
exploded_actors_df.to_csv("clean_data/actors.csv", index=False)
exploded_directors_df.to_csv("clean_data/directors.csv", index=False)


,name,id,gender
0,Tom Hanks,31,2
1,Tim Allen,12898,2
2,Don Rickles,7167,2
3,Jim Varney,12899,2
4,Wallace Shawn,12900,2


,name,id,gender
0,John Lasseter,7879,2
1,Joe Johnston,4945,2
2,Howard Deutch,26502,2
3,Forest Whitaker,2178,2
4,Charles Shyer,56106,2


In [3]:
# Load your credits CSV
df = pd.read_csv("../data/movies_keywords.csv")

# Define a function to extract name, id, and gender from cast JSON
def extract_keyword_info(keyword_json_str):
    try:
        keyword_list = ast.literal_eval(keyword_json_str)
        return [{'name': keyword['name'], 'id': str(keyword['id'])} for keyword in keyword_list]
    except:
        return []

# Apply the function to the 'keywords' column
df['simplified_keywords'] = df['keywords'].apply(extract_keyword_info)

# explode to have one row per cast member (with name, id, gender)
exploded_keywords_df = df.explode('simplified_keywords')
exploded_keywords_df = pd.json_normalize(exploded_keywords_df['simplified_keywords'])

# Show result
display(exploded_keywords_df.head())

# Save the exploded DataFrame to a CSV file
exploded_keywords_df.to_csv("../clean_data/keywords.csv", index=False)


,name,id
0,jealousy,931
1,toy,4290
2,boy,5202
3,friendship,6054
4,friends,9713


The above code downloads, cleans, and merges multiple datasets from the movies database. It starts with the metadata dataset and breaks up the columns that contain objects. It then merges the credits dataset with the metadata dataset, keeping the name, gender, and id of the top 5 cast members as well as the director. It then merges the keywords dataset, keeping the top 5 keywords along with their ids. The final dataset is saved as a CSV file for further analysis. Finally, it explodes the cast, crew, and keyword info into their own datasets so that we have access to the mappings of the ids to the names.